<a href="https://colab.research.google.com/github/theSquaredError/task-planning-with-llm/blob/main/LLMTaskPlanning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch transformers trl accelerate datasets

In [ ]:
!git clone https://github.com/ysymyth/ReAct.git

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [6]:
from openai import OpenAI



client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1/",
    api_key= api_key
)

def chat_completion(prompt, stop=["\n"]):
    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    # model="google/gemma-2-2b-it"
    model = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
    max_tokens=500
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        stream=False,
        stop=stop
    )

    # Extract the text from the response

    return response.choices[0].message.content

# Example usage:
prompt = "What is the square root of -1?"
result = chat_completion(prompt)
print(result)


To determine the square root of -1, I start by recalling that in the realm of real numbers, the square of any real number is always non-negative. Therefore, there is no real number that can be squared to yield -1.




In [ ]:
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

def llm(prompt, stop=["\n"]):
    response = openai.Completion.create(
      model="text-davinci-002",
      prompt=prompt,
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response["choices"][0]["text"]

In [ ]:
!pip install alfworld

In [ ]:
import yaml
import alfworld
import alfworld.agents.environment
with open('ReAct/base_config.yaml') as reader:
    config = yaml.safe_load(reader)

split = "eval_out_of_distribution"

env = getattr(alfworld.agents.environment, config["env"]["type"])(config, train_eval=split)
env = env.init_env(batch_size=1)

def process_ob(ob):
    if ob.startswith('You arrive at loc '):
        ob = ob[ob.find('. ')+2:]
    return ob

In [ ]:
import sys

def alfworld_run(prompt, to_print=True, ob=''):
    init_prompt = prompt + ob + '\n>'
    prompt = ''
    if to_print:
        print(ob)
        sys.stdout.flush()
    for i in range(1, 50):
        action = chat_completion(init_prompt + prompt, stop=['\n']).strip()
        observation, reward, done, info = env.step([action])
        observation, reward, done = process_ob(observation[0]), info['won'][0], done[0]
        if action.startswith('think:'):
            observation = 'OK.'
        if to_print:
            print(f'Act {i}: {action}\nObs {i}: {observation}')
            sys.stdout.flush()
        prompt += f' {action}\n{observation}\n>'
        if done:
            return reward
    return 0

In [ ]:
!pip install --q alfworld[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.2/628.2 kB 33.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.2/289.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.3/356.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.8 MB

In [ ]:
!alfworld-download

Extracting json_2.1.1_json.zip: 100% 7302/7302 [00:06<00:00, 1066.34it/s]
Extracting json_2.1.1_pddl.zip: 100% 7080/7080 [00:02<00:00, 2487.45it/s]
Extracting json_2.1.2_tw-pddl.zip: 100% 4027/4027 [00:02<00:00, 1946.30it/s]


In [ ]:
import yaml
import alfworld
import alfworld.agents.environment.alfred_tw_env
# with open('base_config.yaml') as reader:
#     config = yaml.safe_load(reader)

split = "eval_out_of_distribution"
# config['env']['type'] = 'AlfredTwEnv'
# print(config['env']['type'])
env = getattr(alfworld.agents.environment.alfred_tw_env, 'AlfredTWEnv')(config, train_eval=split)
env = env.init_env(batch_size=1)

def process_ob(ob):
    if ob.startswith('You arrive at loc '):
        ob = ob[ob.find('. ')+2:]
    return ob

Initializing AlfredTWEnv...


100%|██████████| 341/341 [00:00<00:00, 903.34it/s]

Overall we have 134 games in split=eval_out_of_distribution
Evaluating with 134 games


In [ ]:
import numpy as np
# interact
obs, info = env.reset()
print(f'obs: {obs[0]}')
print(f'info: {info}')
done = False
while done==False:
    # get random actions from admissible 'valid' commands (not available for AlfredThorEnv)
    admissible_commands = list(info['admissible_commands']) # note: BUTLER generates commands word-by-word without using admissible_commands
    random_actions = [np.random.choice(admissible_commands[0])]

    # step
    obs, scores, dones, infos = env.step(random_actions)
    print("Action: {}, Obs: {}, done: {}".format(random_actions[0], obs[0], dones[0]))
    done = dones[0]


obs: -= Welcome to TextWorld, ALFRED! =-

You are in the middle of a room. Looking quickly around you, you see a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 3, a countertop 2, a countertop 1, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.

Your task is to: put a clean plate in countertop.
info: {'admissible_commands': [['go to cabinet 1', 'go to cabinet 2', 'go to cabinet 3', 'go to cabinet 4', 'go to cabinet 5', 'go to cabinet 6', 'go to coffeemachine 1', 'go to countertop 1', 'go to countertop 2', 'go to countertop 3', 'go to drawer 1', 'go to drawer 2', 'go to drawer 3', 'go to fridge 1', 'go to garbagecan 1', 'go to microwave 1', 'go to shelf 1', 'go to shelf 2', 'go to shelf 3', 'go to sinkbasin 1', 'go to stoveburner 1', 'go to stoveburner 2', 'go to stov

In [ ]:
prompt1 = 'Environment Description: Interact with a house hold to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. \n Actions You should choose from two actions: “THOUGHT” or “ACTION”. If you choose “THOUGHT,” you should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format: \n THOUGHT: your thoughts \n ACTION: your next action. \n If you choose “ACTION”, you should directly output the action in this turn. Your output must strictly follow this format: \n ACTION: your next action. \n After your each turn, the environment will give you immediate feedback based on which you plan your next few steps. if the environment output Nothing happened, that means the previous action is invalid and you should try more options.'

In [ ]:
print(prompt1)

Environment Description: Interact with a house hold to solve a task. Imagine you are an intelligent agent in a household environment and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given the detailed description of the current environment and your goal to accomplish. For each of your turn, you will be given a list of actions which you can choose one to perform in this turn. 
 Actions You should choose from two actions: “THOUGHT” or “ACTION”. If you choose “THOUGHT,” you should first think about the current condition and plan for your future actions, and then output your action in this turn. Your output must strictly follow this format: 
 THOUGHT: your thoughts 
 ACTION: your next action. 
 If you choose “ACTION”, you should directly output the action in this turn. Your output must strictly follow this format: 
 ACTION: your next action. 
 After your each turn, the environment will give you immediate feedback based on w

In [ ]:
prompt3 = 'Reminder: \n 1. The action must be chosen from the given available actions. Any actions except provided available actions will be regarded as illegal. \n 2. Think when necessary, try to act directly more in the process.\n Initial Observation You are in the middle of a room. Looking quickly around you, you see a cabinet 16, a cabinet 15, a cabinet 14, a cabinet 13, a cabinet 12, a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 2, a countertop 1, a diningtable 1, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a safe 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1. Your task is to: '

In [ ]:
print(prompt3)

Reminder: 
 1. The action must be chosen from the given available actions. Any actions except provided available actions will be regarded as illegal. 
 2. Think when necessary, try to act directly more in the process.
 Initial Observation You are in the middle of a room. Looking quickly around you, you see a cabinet 16, a cabinet 15, a cabinet 14, a cabinet 13, a cabinet 12, a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 2, a countertop 1, a diningtable 1, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a safe 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1. Your task is to: 


In [ ]:
prefixes = {
    'pick_and_place': 'put',
    'pick_clean_then_place': 'clean',
    'pick_heat_then_place': 'heat',
    'pick_cool_then_place': 'cool',
    'look_at_obj': 'examine',
    'pick_two_obj': 'puttwo'
}
cnts = [0] * 6
rs = [0] * 6

for _ in range(134):
    ob, info = env.reset()
    ob = '\n'.join(ob[0].split('\n\n')[1:])
    name = '/'.join(info['extra.gamefile'][0].split('/')[-3:-1])
    print(name)
    for i, (k, v) in enumerate(prefixes.items()):
        if name.startswith(k):
            prompt = prompt1+prompt3
            print(k, v)
            r = alfworld_run(prompt, ob=ob)
            rs[i] += r
            cnts[i] += 1
            break
    print(_+1, 'r', r, 'rs', rs, 'cnts', cnts, 'sum(rs)/sum(cnts)', sum(rs) / sum(cnts))
    print('------------\n')

pick_cool_then_place_in_recep-Mug-None-Cabinet-10/trial_T20190909_121559_082363
pick_cool_then_place cool
You are in the middle of a room. Looking quickly around you, you see a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 3, a countertop 2, a countertop 1, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.
Your task is to: put a cool mug in cabinet.
Act 1: THOUGHT: I need to find a cool mug and then get it to the cabinet.
Obs 1: Nothing happens.
Act 2: ACTION: Search for a cool mug.
Obs 2: Nothing happens.
Act 3: Thought: I should try to check if there's a cool mug in one of the available places.
Obs 3: Nothing happens.
Act 4: > THOUGHT: I should  try "ACTION"  by looking at the other location to see if I can find a cool mug there.
Obs 4: Nothing happens.
Act 5: ACT

KeyboardInterrupt: 